In [ ]:
#!/usr/bin/env python
# coding: utf-8

In[1]:

In [ ]:
get_ipython().magic('pylab nbagg')

# Import simulation parameters from web exported JSON file

In[2]:

In [ ]:
import os
import json
from tvb.simulator.lab import *
LOG = get_logger('demo')
from tvb.datatypes import noise_framework
from tvb.datatypes.equations import HRFKernelEquation
from tvb.basic.traits.parameters_factory import *

In [ ]:
available_models = get_traited_subclasses(models.Model)
available_monitors = get_traited_subclasses(monitors.Monitor)
available_integrators = get_traited_subclasses(integrators.Integrator)
available_couplings = get_traited_subclasses(coupling.Coupling)

Having a path towards the exported JSON, we will: <br>
- read the simulation parameters as strings, <br>
- transform them into objects, <br>
- run the simulation, and <br>
- plot some timeseries results at the end.<br>
<br>
Let us start by reading the JSON from a file path.

In[14]:

In [ ]:
with open('tvb_simulation_3.json', 'r') as fdr:
    with open('tvbsim.json', 'w') as fdw:
        json.dump(json.loads(json.load(fdr)['_simulator_configuration']), fdw, indent=1)

In[27]:

In [ ]:
json_path = "tvb_simulation_3.json"

In [ ]:
def ignore_key(key): # v1.4 -> 1.5 compat
    keys = ['_noise', 'pre_expr', 'post_expr']
    return any(k in key for k in keys)

In [ ]:
with open(json_path, 'rb') as input_file:
    simulation_json = input_file.read()
    simulation_json = json.loads(simulation_json)
    simulation_config = {}
    for key, val in json.loads(simulation_json["_simulator_configuration"]):
        nonempty_key = len(key) > 1
        no_noise_v14 = '_noise' not in key
        no_pre_expr_v141 = key.startswith('monitors') and not key.endswith('pre_expr')
        no_post_expr_v141 = key.startswith('monitors') and not key.endswith('post_expr')
        if nonempty_key and no_pre_expr_v141 and no_post_expr_v141:
            simulation_config[key, valu]

Next, we will transform the JSON into simulator params.<br>
<br>
We do that by first grouping parameters over configuration class (model, integrator, coupling, etc.)

In[28]:

In [ ]:
model_key = 'model'
model_name = simulation_config[model_key]
#noise_key = '%s%s%s%s_noise' % (model_key, KEYWORD_PARAMS, KEYWORD_OPTION, model_name)
#noise_name = simulation_config[noise_key]
#random_stream_key = '%s%s%s%s_random_stream' % (noise_key, KEYWORD_PARAMS, KEYWORD_OPTION, noise_name)

In [ ]:
selectors = ['coupling', 'integrator']
selectors.append(model_key)
#selectors.append(noise_key)
#selectors.append(random_stream_key)
                                    
transformed = collapse_params(simulation_config, selectors)

Next we parse the string values (from strings into float, arrays, etc.):

In[29]:

In [ ]:
converted = {str(k): try_parse(v) for k,v in transformed.items() }

Now, let us build simulator configuration class instances (Model, Integrator, etc.):

In[30]:

In [ ]:
model_parameters = converted['model_parameters']
integrator_parameters = converted['integrator_parameters']
coupling_parameters = converted['coupling_parameters']

TODO: this parameter shuld be correctly parsed and considered:

In [ ]:
del model_parameters['state_variable_range_parameters']

In [ ]:
noise_framework.build_noise(model_parameters)
noise_framework.build_noise(integrator_parameters)

In [ ]:
model_instance = available_models[converted['model']](**model_parameters)
integr_instance = available_integrators[converted['integrator']](**integrator_parameters)
coupling_inst = available_couplings[converted['coupling']](**coupling_parameters)
conduction_speed = converted['conduction_speed']

TODO: reloading the original Connectivity ...<br>
TODO: detect surface simulations and reconfigure ...

In [ ]:
conn = connectivity.Connectivity(load_default=True)
model_instance

The monitors will require special care, as their number can vary.<br>
<br>
Thus we build monitor instances in a loop.

In[20]:

In [ ]:
monitors = converted['monitors']
monitors_parameters = converted['monitors_parameters']
monitors_list = []

In [ ]:
for monitor_name in monitors:
    if (monitors_parameters is not None) and (str(monitor_name) in monitors_parameters):
        current_monitor_parameters = monitors_parameters[str(monitor_name)]
        HRFKernelEquation.build_equation_from_dict('hrf_kernel', current_monitor_parameters, True)
        monitors_list.append(available_monitors[str(monitor_name)](**current_monitor_parameters))
    else:
        ### We have monitors without any UI settable parameter.
        monitors_list.append(available_monitors[str(monitor_name)]())

We are now ready to configure the Simulator object with all parameter instances previously built:

In[21]:

In [ ]:
sim = simulator.Simulator(connectivity=conn, coupling=coupling_inst, 
                          model=model_instance, integrator=integr_instance,
                          monitors=monitors_list, conduction_speed=conduction_speed)
sim.configure()

Next we can actually run the simulation, for the length specified in the JSON:

In[9]:

In [ ]:
result_data = {m_name: [] for m_name in monitors}
result_time = {m_name: [] for m_name in monitors}

In [ ]:
simulation_length = converted['simulation_length']

In [ ]:
for result in sim(simulation_length=simulation_length):
    for j, monitor_name in enumerate(monitors):
        if result[j] is not None:
            result_time[monitor_name].append(result[j][0])
            result_data[monitor_name].append(result[j][1])

We will plot resulting timeseries in a generic manner.<br>
<br>
Different monitors can have multiple State Variables and Modes.

In[10]: